Esse arquivo tem como intuito tratar, 'plotar' e avaliar os datasets presentes a pasta Saneamento/datasets com o fim de compor a base metodológica do trabalho semestral de RP2 "Uma análise quantitativa da distribuição de casos de dengue na cidade de São Paulo em suas várias granularidades."

Para melhor aproveitamento desse arquivo use Python 3.

In [ ]:
# importando bibliotecas necessparias.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns